In [64]:
import scipy.io as sio
import numpy as np
from os import getcwd
from os.path import join 
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.multivariate.pca import PCA
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd 

In [65]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [69]:
fname = join(os.getcwd(),'data','data3SS2009.mat')
mat_contents = sio.loadmat(fname)
data = mat_contents['dataset']

N, Chno, Nc = dataset.shape
# N: number of samples
# Chno: number of channels
# Nc: number of cases

y = mat_contents['labels'].reshape(Nc)


In [71]:
Ch1 = dataset[:,0,:] # célula de carga: força do shaker
Ch2 = dataset[:,1,:] # acelerômetro: base
Ch3 = dataset[:,2,:] # acelerômetro: 1o andar
Ch4 = dataset[:,3,:] # acelerômetro: 2o andar
Ch5 = dataset[:,4,:] # acelerômetro: 3o andar

Ts = 3.125 * 1e-3 # sampling time
time = (np.linspace(1,N,N) - 1) * Ts




In [48]:
ARmodel_2 = AutoReg(Ch2[:,1],30,old_names=False).fit()
ARmodel_3 = AutoReg(Ch3[:,1],30,old_names=False).fit()
ARmodel_4 = AutoReg(Ch4[:,1],30,old_names=False).fit()
ARmodel_5 = AutoReg(Ch5[:,1],30,old_names=False).fit()
vec_2 = ARmodel_2.params
vec_3 = ARmodel_3.params
vec_4 = ARmodel_4.params
vec_5 = ARmodel_5.params
vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))

ARmodel_var = vec_temp

for i in range(1,850):
    ARmodel_2 = AutoReg(Ch2[:,i],30,old_names=False).fit()
    ARmodel_3 = AutoReg(Ch3[:,i],30,old_names=False).fit()
    ARmodel_4 = AutoReg(Ch4[:,i],30,old_names=False).fit()
    ARmodel_5 = AutoReg(Ch5[:,i],30,old_names=False).fit()
    vec_2 = ARmodel_2.params
    vec_3 = ARmodel_3.params
    vec_4 = ARmodel_4.params
    vec_5 = ARmodel_5.params
    vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))
    ARmodel_var = np.vstack([ARmodel_var,vec_temp])

In [59]:
## TEST SVM with kernel Linear
X = ARmodel_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision : {accuracy * 100:.2f}%")

Précision : 100.00%


In [60]:
## TEST SVM with RBM (Radial Basis Fuction) and C=10
model = SVC(kernel='rbf', C=1.0, gamma='scale')  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision : {accuracy * 100:.2f}%")

Précision : 69.02%


In [61]:
## TEST SVM with polynomial 
model = SVC(kernel='poly', C=1.0, degree=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision : {accuracy * 100:.2f}%")

Précision : 94.12%


In [62]:
## Test kNN with k=3
from sklearn.neighbors import KNeighborsClassifier

k = 3
model = KNeighborsClassifier(n_neighbors=k)
X = ARmodel_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision : {accuracy * 100:.2f}%")

Précision : 97.25%


In [63]:
## Test kNN , k=5 and Manhattan metric

k = 5  
metric = 'manhattan' 
model = KNeighborsClassifier(n_neighbors=k, p=1, metric=metric)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision : {accuracy * 100:.2f}%")

Précision : 99.22%


In [ ]:
# The choice between an SVM model with a non-linear kernel and a linear model 
#will depend on the complexity of the data and the problem we're trying to solve.
#For non-linear data, SVMs with non-linear kernels can offer superior performance, 
#while linear models are more appropriate for simple relationships and linear data. 
#This is the case here, as we have an accuracy of 100%.